# Lakehouse Backup
This notebook makes a regular backup of the tables from FUAM_Lakehouse to the FUAM_Lakehouse_Backup 

In [ ]:
import sempy.fabric as fabric
from datetime import datetime, timedelta

In [ ]:
backup_files = True
keep_historic_days = 7

fuam_workspace_id = 'eb764c8b-cf3b-55be-adf4-348fe9233657'
fuam_lakehouse_id = '729eb8a2-8070-5ed8-ad43-dccbc00b32af'

In [ ]:
# Init the client
client = fabric.FabricRestClient()

# Set date helpers
current_time = datetime.now()

In [ ]:
fuam_lakehouse_tables_path = f"abfss://{fuam_workspace_id}@onelake.dfs.fabric.microsoft.com/{fuam_lakehouse_id}/Tables"
fuam_lakehouse_files_path = f"abfss://{fuam_workspace_id}@onelake.dfs.fabric.microsoft.com/{fuam_lakehouse_id}/Files"


In [ ]:
notebookutils.fs.fastcp(fuam_lakehouse_tables_path, 'Files/'+ current_time.strftime("%Y/%m/%d") + '/Tables', True)
if backup_files:
   notebookutils.fs.fastcp(fuam_lakehouse_files_path, 'Files/'+ current_time.strftime("%Y/%m/%d") + '/Files', True) 

Remove historic backups olders than keep_historic_days


In [ ]:
subfolders = []
def list_subfolders(path, max_level, date_path):
  for item in mssparkutils.fs.ls(path):
    if max_level > 0:
      list_subfolders(item.path, max_level - 1, date_path + '-' + item.name)
    else:
      fold = {}
      fold["date"] = datetime.strptime((date_path + '-' + item.name)[1:], "%Y-%m-%d")
      fold["path"] = item.path
      subfolders.append(fold)
    

list_subfolders('Files/', max_level= 2, date_path = '')


for dat in subfolders:
  
  time_between_insertion = datetime.now() - dat['date']
  if  time_between_insertion.days > keep_historic_days:
    print(dat['date'])
    print(dat['path'])
    mssparkutils.fs.rm(dat['path'], True) # Set the last parameter as True to remove all files and directories recursively

Remove empty folders after old backups have been removed

In [ ]:
def find_empty_dirs(path, max_level):
    directories = notebookutils.fs.ls(path)
    for directory in directories:
        if (directory.size == 0) & (max_level > 0):
            find_empty_dirs(directory.path, max_level - 1)
            contents = notebookutils.fs.ls(directory.path)
            if len(contents) == 0:
                # Logic
                notebookutils.fs.rm(directory.path, recurse=True)
                print(f"Removed empty directory: {directory.path}")

find_empty_dirs('Files/', 4)